In [ ]:
import sys
!{sys.executable} -m pip install transformers
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install torchvision

In [ ]:

import torch
from transformers import AutoTokenizer, AutoModel



tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")  #multillingual
model = AutoModel.from_pretrained("bert-base-multilingual-cased")
print("done")

In [ ]:
def wordextractor(sentence, number): #split() helyette
    words=sentence.split(" ")
    return words[number]

print(wordextractor("A kutyeszok alszalanak", 2))


In [ ]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from torchvision import datasets
import torch
import torch.nn as nn
import torch.optim as opti
import csv

In [ ]:
def casetonumber(case):
    switch = {
        "Nom": 1,
        "Acc": 2,
        "Ins": 3,
        "Ine": 4,
        "Sup": 5,
        "Sub": 6
    }
    return switch.get(case, "Invalid case")

In [18]:
file1 = open("train.tsv")
train_tsv=csv.reader(file1, delimiter="\t")
file2 = open("test.tsv")
test_tsv=csv.reader(file2, delimiter="\t")
training_data=[]
test_data=[]
i=0
for obj in train_tsv:
    training_data.append([])
    training_data[i].append(obj[0])
    training_data[i].append(obj[2])
    training_data[i].append(casetonumber(obj[3]))
    i=i+1

i=0
for obj in test_tsv:
    test_data.append([])
    test_data[i].append(obj[0])
    test_data[i].append(obj[2])
    test_data[i].append(casetonumber(obj[3]))
    i=i+1


trainloader=torch.utils.data.DataLoader(training_data, batch_size=128, shuffle=True)
testloader=torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=True)
file1.close()
file2.close()
print("DONE")

DONE


In [17]:
print(testloader)